<a href="https://colab.research.google.com/github/SWEN90006/tutorials/blob/main/SWEN90006_Tutorial_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SWEN90006 Tutorial 5

## Introduction
Encapsulation is an abstraction mechanism that aids in programming, but
that adds complexity to testing. We often need to break the information
hiding utilised by classes in order to examine the class state for the
testing purposes.

The aim of this tutorial is for you to explore some of the issues in
object oriented testing through the simple Graph given below. The graph
uses an adjacency matrix that records which vertices are "*adjacent*" in
the graph.

## Working With the Program
### Prepare the Java Kernel
Since Java is not natively supported by Colab, we need to run the following code to enable Java kernel on Colab.

1. Run the cell bellow (click it and press Shift+Enter),
2. Refresh the Notebook (F5)
3. Change the kernel to Java (Runtime -> Change Runtime Type -> Java)

In [ ]:
!wget https://github.com/SpencerPark/IJava/releases/download/v1.3.0/ijava-1.3.0.zip
!unzip ijava-1.3.0.zip
!python install.py --sys-prefix

In [ ]:
%%loadFromPOM

<dependency>
  <groupId>junit</groupId>
  <artifactId>junit</artifactId>
  <version>4.13.2</version>
</dependency>

The following is a basic Java implementation of `Graph`.

In [ ]:
import java.util.*;

public class Graph {
    
    //------------ Private Attributes ----------------------------
    //    The representation of a graph consists of an array
    //    of vertices that map nodes (just integers) to array 
    //    indexes. The adjacency matrix _matrix sets matrix[i][j]
    //    to true if there is an edge between _vertices[i] and 
    //    _vertices[j].
    //  
    //    The operations must maintain the following invariant:
    //        _vertices[i] is defined iff i < _allocated
    //        _allocated <= _order
    static final String EMPTY_GRAPH = "Empty Graph";
    
    private int     _order;      // The number of vertices allowed 
    private int     _allocated;  // The next free space in the vertex array
    private int     _vertices[]; // A list of the actual vertices
    private boolean _matrix[][]; // The adjacency matrix
    
    public Graph(int n) {
        // Create a matrix of size n and initial all of the 
        // state variables so that the invariant is maintained.
        _order = n;
        _allocated = 0;
        _vertices = _vertices(n);
        _matrix = _allocate(n);
    }
    
    public Graph(int n, int allocated, int[] vertices, boolean matrix[][]) {
        _order = n;
        _allocated = allocated;
        _vertices = vertices;
        _matrix = matrix;
     }
    
    private static boolean[][] _allocate(int n) {
        return new boolean[n][n];
    }
    
    private static int[] _vertices(int n) {
        return new int[n];
    }
    
    private int _lookup(int m) {
        int index = 0;
        while (index < _allocated && _vertices[index] != m) {
            index = index + 1;
        }

        if (index == _allocated) {
            return _order + 1;
        }
        else {
            return index;
        }
    }
    
    public void addVertex(int v) {
        // Add a vertex to the vertex graph. For the moment we
        // assume that vertices are just integers.
        if (_allocated < _order) {
            _vertices[_allocated] = v;
            _allocated = _allocated + 1;
        }
    }
    
    public void addEdge(int m, int n) {
        // Add an edge to the graph. Edges are specified by pairs
        // of vertices. To add the edge correctly it is necessary
        // that m and n have already been added to graph as vertices.

        int mIndex = _lookup(m);
        int nIndex = _lookup(n);

        if (mIndex < _order && nIndex < _order) {
            _matrix[mIndex][nIndex] = true;
            _matrix[nIndex][mIndex] = true;
        }
    }
    
    public void deleteVertex(int v) {
        // We can only delete a node if it is not part of some edge
        // in the graph and it exists as an actual vertex in the 
        // graph.
        
        int vIndex = _lookup(v);
    
        if (vIndex < _order) {
            boolean isEdge = false;    
            for (int i = 0; i < _allocated; i++) {
                isEdge = isEdge || _matrix[vIndex][i] || _matrix[i][vIndex];
            }
    
            if (!isEdge) {
                for (int i = vIndex; i < _allocated-1; i++) {
                    _vertices[i] = _vertices[i+1];
                }
            }
            _allocated = _allocated - 1;
        }
    }
    
    public void deleteEdge(int m, int n) {
        // We can only delete an edge if the two specified
        // vertices are in the graph.
        
        int mIndex = _lookup(m);
        int nIndex = _lookup(n);
    
        if (mIndex < _order && nIndex < _order) 
            _matrix[mIndex][nIndex] = false;
    }
    
    // Overrides ToString()
    public String toString() {
        StringBuilder s = new StringBuilder();
        if (_allocated == 0)
        {
            return EMPTY_GRAPH;
        }
        else {
            s.append("\n   ");
            for (int i = 0; i < _allocated; i++)
            {
                 s.append(i + " ");
            }
            s.append("\n");
            for (int i = 0; i < _allocated; i++) {
                 s.append(i + ": ");
                 for (boolean j : _matrix[i]) {
                      s.append((j ? "T" : " ") + " ");
                 }
                 s.append("\n");
            }
            return s.toString();
        }
   }
}

The code block below demostrates how to create graph

In [ ]:
// Method 1: pass all params into the constructor
// Pre-define a matrix
boolean[][] matrix = new boolean[6][6];
for (int i = 0; i < matrix.length ; i++) {
    Arrays.fill(matrix [i], false);
}

matrix[1][3] = true;
matrix[3][1] = true;

// Graph(int n, int allocated, int[] vertices, boolean matrix[][])
// int n: The number of vertices allowed 
// int allocated: The next free space in the vertex array
// int[] verties: A list of the actual vertices
// boolean matrix[][]: The adjacency matrix
Graph graph_1 = new Graph(6, 6, new int[] {0, 1, 2, 3, 4, 5}, matrix);

// Print the Graph
System.out.println(graph_1);

// Method 2:
Graph graph_2 = new Graph(6);
System.out.println(graph_2);
System.out.println();
// Add Vertex
graph_2.addVertex(0);
graph_2.addVertex(1);
graph_2.addVertex(2);
graph_2.addVertex(3);
System.out.println(graph_2);
graph_2.addVertex(4);
graph_2.addVertex(5);

// Add Edge
graph_2.addEdge(1, 3);

// Print the Graph
System.out.println(graph_2);

// Check if they are identical or not
// In this case, it should be true
System.out.println(graph_1.toString().equals(graph_2.toString()))


   0 1 2 3 4 5 
0:             
1:       T     
2:             
3:   T         
4:             
5:             

Empty Graph


   0 1 2 3 
0:             
1:             
2:             
3:             


   0 1 2 3 4 5 
0:             
1:       T     
2:             
3:   T         
4:             
5:             

true


In [ ]:
import java.util.Arrays;
import java.util.Collection;

import org.junit.Test;
import org.junit.runner.JUnitCore;
import org.junit.runner.Result;
import org.junit.runner.RunWith;
import org.junit.runner.notification.Failure;
import org.junit.runners.Parameterized;

import junit.framework.TestCase;

@RunWith(Parameterized.class)
public class TestGraph extends TestCase {
    @Parameterized.Parameter(0)
    public String actual;
    @Parameterized.Parameter(1)
    public String expected;
    
    @Parameterized.Parameters(name = "{index}:\ngraph 1: \n{0}\ngraph 2:\n{1}\n")
    public static Collection<Object[]> data() {

        // Define your params and constructor here
        boolean[][] matrix = new boolean[6][6];
        for (int i = 0; i < matrix.length ; i++) {
            Arrays.fill(matrix [i], false);
        }
        
        matrix[1][3] = true;
        matrix[3][1] = true;
        
        Graph expectedGraph = new Graph(6, 6, new int[]{0, 1, 2, 3, 4, 5}, matrix);

        Graph actualGraph = new Graph(6);
        actualGraph.addVertex(0);
        actualGraph.addVertex(1);
        actualGraph.addVertex(2);
        actualGraph.addVertex(3);
        actualGraph.addVertex(4);
        actualGraph.addVertex(5);
        actualGraph.addEdge(1, 3);
        
        Graph emptyGraph = new Graph(6);
        // End of pre-define
        
        
        Object[][] data = new Object[][]{
            // Your Test cases start here
            // Please follow the pattern: expected graph, actual graph

            // Success
            {expectedGraph.toString(), actualGraph.toString()},
            // Success
            {Graph.EMPTY_GRAPH, emptyGraph.toString()},
            // Fail
            {Graph.EMPTY_GRAPH, actualGraph.toString()}
            // Your Test cases end here
            };
        return Arrays.asList(data);
    }
    
    @Test
    public void testGraph() {
        assertEquals(expected, actual);
    }
}

In [ ]:
Result result = JUnitCore.runClasses(TestGraph.class);
for (Failure failure : result.getFailures()) {
     System.out.println(failure.toString());
}
System.out.println(String.format("Total run count: %s, Failed run count: %s", result.getRunCount(), result.getFailureCount()));

testGraph[2:
graph 1: 
Empty Graph
graph 2:

   0 1 2 3 4 5 
0:             
1:       T     
2:             
3:   T         
4:             
5:             

](REPL.$JShell$44H$TestGraph): expected:<[
   0 1 2 3 4 5 
0:             
1:       T     
2:             
3:   T         
4:             
5:             
]> but was:<[Empty Graph]>
Total run count: 3, Failed run count: 1


## Your Tasks

### Task 1
Consider the `addEdge` and `deleteEdge` methods in the `Graph` class
above. Derive test cases for path coverage and condition coverage for
these two methods. **Note** that it may be necessary to examine the
state variables in your test cases. Sketch how you would achieve this.

### Task 2
Given that a Graph object has already been initialized with the number K, draw a finite state automaton for this `Graph` object. **Note** that it is not
always possible to add an edge and it is not always possible to delete a
vertex. You will need to consider the states and the guards on
transitions to ensure all of the conditions.

### Task 3
Derive a set of test cases to test every transition in your graph.

